In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, ArrayType, FloatType

In [2]:
spark = SparkSession.builder \
.master("local[2]") \
.appName("Write to disk") \
.config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1") \
.getOrCreate()

:: loading settings :: url = jar:file:/opt/manual/spark-3.1.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-42d6f5a2-665e-4116-84a6-e665523d6f26;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.1.1/spark-avro_2.12-3.1.1.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-avro_2.12;3.1.1!spark-avro_2.12.jar (165ms)
:: resolution report :: resolve 4908ms :: artifacts dl 178ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.1.1 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicte

In [3]:
# ! wget -O /home/train/datasets/Hotel_Reviews.csv.gz \
# https://github.com/erkansirin78/datasets/raw/master/Hotel_Reviews.csv.gz

# Read Data

In [3]:
df = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/Hotel_Reviews.csv.gz") \
.withColumn("Review_Date", F.to_date("Review_Date", "M/d/y")) \
.withColumn("Tags", F.split(F.col("Tags"), ",").cast(ArrayType(StringType())))

In [4]:
df.limit(2).toPandas()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[[' Leisure trip ', ' Couple ', ' Duplex Dou...",0 days,52.3605759,4.9159683
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[[' Leisure trip ', ' Couple ', ' Duplex Dou...",0 days,52.3605759,4.9159683


# Write to local disk csv format

In [21]:
df.withColumn("Tags", F.col("Tags").cast(StringType())) \
.repartition(2) \
.write.format("csv") \
.option("header", True) \
.option("sep", ",") \
.mode("overwrite") \
.save("file:///home/train/spark_out/hotel_csv")

In [22]:
df_source = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/spark_out/hotel_csv") \
.withColumn("Review_Date", F.to_date("Review_Date", "y-M-d")) \
.withColumn("Tags", F.split(F.col("Tags"), ",").cast(ArrayType(StringType())))

In [23]:
df_source.limit(2).toPandas()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,163 Marsh Wall Docklands Tower Hamlets London ...,2682,2017-07-03,7.1,Britannia International Hotel Canary Wharf,Israel,Room wifi should be provided for free And TV s...,17,9086,The room interior decoration was nice and was ...,16,3,10.0,"[[[' Leisure trip ', ' Couple ', ' Standar...",31 days,51.5019097,-0.0232208
1,Wilton Place Knightsbridge Westminster Borough...,70,2016-08-12,9.1,The Berkeley,Jersey,My room was on the 6th floor at the end of the...,123,232,The room and staff were nice,7,40,6.3,"[[[' Business trip ', ' Solo traveler ', '...",356 day,51.5020912,-0.1577795


# parquet

In [24]:
df \
.repartition(2) \
.write \
.mode("overwrite") \
.save("file:///home/train/spark_out/hotel_parquet")

In [25]:
df_from_parquet = spark.read.load("file:///home/train/spark_out/hotel_parquet")

In [26]:
df_from_parquet.limit(2).toPandas()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,Plaza Paisos Catalans s n Sants Montju c 08014...,420,2016-05-26,8.6,Barcel Sants,Australia,Poor design of bathroom and toilet area,8,4379,Great location Loved the space theme,7,38,7.1,"[[' Leisure trip ', ' Couple ', ' Deluxe Dou...",434 day,41.3798964,2.1412433
1,1 Addington Street Lambeth London SE1 7RY Unit...,1322,2016-12-31,8.4,Park Plaza County Hall London,United Kingdom,Could of got a better price booking direct,9,6117,Location,2,2,9.2,"[[' Leisure trip ', ' Family with young child...",215 day,51.5014001,-0.1160092


In [27]:
df_from_parquet.printSchema()

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: date (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



# ORC

In [28]:
df \
.repartition(2) \
.write \
.mode("overwrite") \
.orc("file:///home/train/spark_out/hotel_orc")

In [29]:
df_from_orc = spark.read.orc("file:///home/train/spark_out/hotel_orc")

In [30]:
df_from_orc.limit(2).toPandas()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,Plaza Paisos Catalans s n Sants Montju c 08014...,420,2016-05-26,8.6,Barcel Sants,Australia,Poor design of bathroom and toilet area,8,4379,Great location Loved the space theme,7,38,7.1,"[[' Leisure trip ', ' Couple ', ' Deluxe Dou...",434 day,41.3798964,2.1412433
1,1 Addington Street Lambeth London SE1 7RY Unit...,1322,2016-12-31,8.4,Park Plaza County Hall London,United Kingdom,Could of got a better price booking direct,9,6117,Location,2,2,9.2,"[[' Leisure trip ', ' Family with young child...",215 day,51.5014001,-0.1160092


In [31]:
df_from_orc.printSchema()

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: date (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



# JSON

In [32]:
df \
.repartition(2) \
.write.format("json") \
.mode("overwrite") \
.save("file:///home/train/spark_out/hotel_json")

In [33]:
df_from_json = spark.read.json("file:///home/train/spark_out/hotel_json")

In [34]:
df_from_json.limit(2).toPandas()

,Additional_Number_of_Scoring,Average_Score,Hotel_Address,Hotel_Name,Negative_Review,Positive_Review,Review_Date,Review_Total_Negative_Word_Counts,Review_Total_Positive_Word_Counts,Reviewer_Nationality,Reviewer_Score,Tags,Total_Number_of_Reviews,Total_Number_of_Reviews_Reviewer_Has_Given,days_since_review,lat,lng
0,420,8.6,Plaza Paisos Catalans s n Sants Montju c 08014...,Barcel Sants,Poor design of bathroom and toilet area,Great location Loved the space theme,2016-05-26,8,7,Australia,7.1,"[[' Leisure trip ', ' Couple ', ' Deluxe Dou...",4379,38,434 day,41.3798964,2.1412433
1,1322,8.4,1 Addington Street Lambeth London SE1 7RY Unit...,Park Plaza County Hall London,Could of got a better price booking direct,Location,2016-12-31,9,2,United Kingdom,9.2,"[[' Leisure trip ', ' Family with young child...",6117,2,215 day,51.5014001,-0.1160092


In [35]:
df_from_json.printSchema()

root
 |-- Additional_Number_of_Scoring: long (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Address: string (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: long (nullable = true)
 |-- Review_Total_Positive_Word_Counts: long (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Total_Number_of_Reviews: long (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: long (nullable = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



# Avro

In [4]:
df \
.repartition(2) \
.write.format("avro") \
.mode("overwrite") \
.save("file:///home/train/spark_out/hotel_avro")

In [6]:
df_from_avro = spark.read.format("avro").load("file:///home/train/spark_out/hotel_avro")

In [7]:
df_from_avro.limit(2).toPandas()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,Plaza Paisos Catalans s n Sants Montju c 08014...,420,2016-05-26,8.6,Barcel Sants,Australia,Poor design of bathroom and toilet area,8,4379,Great location Loved the space theme,7,38,7.1,"[[' Leisure trip ', ' Couple ', ' Deluxe Dou...",434 day,41.3798964,2.1412433
1,1 Addington Street Lambeth London SE1 7RY Unit...,1322,2016-12-31,8.4,Park Plaza County Hall London,United Kingdom,Could of got a better price booking direct,9,6117,Location,2,2,9.2,"[[' Leisure trip ', ' Family with young child...",215 day,51.5014001,-0.1160092


In [8]:
df_from_avro.printSchema()

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: date (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



In [15]:
spark.stop()